In [10]:
import requests
import urllib.request
import urllib.parse
import urllib.error
from bs4 import BeautifulSoup
import ssl
import json
import pandas as pd
from selenium import webdriver
from selenium.webdriver.common.action_chains import ActionChains
import time
import os
import re
from urllib.request import urlopen
import numpy as np
from urllib.request import Request, urlopen
import nltk
from nltk.corpus import stopwords
import unicodedata
nltk.download('stopwords')
from scrapy.http import TextResponse
import datetime
from scrapy import Selector

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\robert.lowe\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [2]:
link = 'https://www.bbcgoodfood.com/recipes/you-it-katsu-curry'
req = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
webpage = urlopen(req).read()
soup = BeautifulSoup(webpage, "html.parser")

In [3]:
def remove_hover_link(soup):
    article= soup.find_all('article', class_='node node-glossary-item node-teaser node-teaser clearfix main row grid-padding')
    for a in article:
        a.p.string= ''
    recipes = []
    for p in soup.findAll("li", class_= "ingredients-list__item"):
        if p.findAll("article", class_='node node-glossary-item node-teaser node-teaser clearfix main row grid-padding'):
            pass
        recipes.append(p.get_text())
    ingredients = []   
    for rep in recipes:   
      a = re.sub(r'\(.*\)', '',rep)
      ingredients.append(a)
    clean = []
    for x in ingredients:   
        cleaner = re.sub(",.*$", "", x)
        clean.append(cleaner)
    return ", ".join(clean)


In [4]:
remove_hover_link(soup)

'4 tbsp rapeseed oil or vegetable oil Rapeseed oil , 2 x 200g pouches cooked rice  Rice r-eye-s , ½ large cucumber, handful mint leaves or coriander leaves, lime wedges, 1 tbsp rapeseed oil or vegetable oil Rapeseed oil , 2 onions, 2 large carrots, 2 garlic cloves, thumb-sized piece ginger, 1 tbsp curry powder, ½ tsp ground turmeric, 400ml can coconut milk Coconut milk ko-ko-nut mill-k , 2 tsp maple syrup  Maple syrup may-pul sir-rup , 1 tbsp cornflour, 8 chicken mini fillets, 200g breadcrumbs '

In [5]:
def bbc_goodfood_scraper(link):
    
    
    '''Code to scrape bbc good food recipes'''
    try:
        # Get recipe
        
        #define user agent and use BeautifulSoup to open web link
        req = Request(link, headers={'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/70.0.3538.102 Safari/537.36'})
        webpage = urlopen(req).read()
        soup = BeautifulSoup(webpage, "html.parser")

        # Get title
        title = soup.find("h1", class_="recipe-header__title")
        file_name = title.text + ".txt"
        file = open(file_name, "w")

        # rating
        stars = soup.findAll("span", class_="rate-fivestar-btn-filled")
        temp = str(len(stars))
        rating = temp.replace("\n", "")
        file.write(rating + "\n")

        # Skill level
        skill = soup.find("section", class_="recipe-details__item--skill-level")
        temp = skill.get_text()
        level = temp.replace("\n", "").replace(" ", "")
        file.write("Difficulty: " + level + "\n")

        # Serves
        serve = soup.find("section", class_="recipe-details__item--servings")
        temp = serve.get_text()
        serving = temp.replace("\n", "").replace(" ", "").replace("Serves", "")
        file.write("Serves: " + serving + "\n")

        # prep time
        prep = soup.find("span", class_="recipe-details__cooking-time-prep")
        temp = prep.get_text()
        prep_time = temp.replace("\n", "")
        file.write(prep_time + "\n")

        # cooking time
        cook = soup.find("span", class_="recipe-details__cooking-time-cook")
        temp = cook.get_text()
        cook_time = temp.replace("\n", "")
        file.write(cook_time + "\n")


        # list comprehension for ingredients 


        ######################################################### method and nutrition not needed#########################################
        # method
        #print("Method")
        file.write("\n" + "Method" + "\n")
        method_counter = 0
        for li in soup.findAll("li", class_="method__item"):
            method_counter += 1
            method_item = li.get_text().replace("\n", "")
        method = [li.get_text().replace("\n", "") for li in soup.findAll("li", class_="method__item")]
           # return method_item
           # file.write(unicodedata.normalize('NFKD', method_item).encode('ascii', 'ignore') + "\n" + "\n")

        # Nutrition
        file.write("Nutrition \n")
        nutrition = soup.find("ul", class_="nutrition")
        for li in nutrition.findAll("li"):
            label = li.find("span", class_="nutrition__label")
            value = li.find("span", class_="nutrition__value")
            #return label.get_text() + ": " + value.get_text()
            file.write(label.get_text() + ": " + value.get_text() + "\n")

      ######################################################### method and nutrition not needed#########################################


        # Create a dataframe
        d = {
            'Title': title.text, 
             "Rating": rating, 
             "Difficulty":  level, 
             "Serving": serving, 
             "Preptime": prep_time, 
             'Cook_time': cook_time, 
             'Ingredients': remove_hover_link(soup),
            #'Method': " ".join(method), 
             #"nutr": label.get_text() + ": " + value.get_text(),
             "Link": link

            }
        df = pd.DataFrame(data=d, index=[0])

        return df
    
    except:
        pass


In [6]:
bbc_goodfood_scraper('https://www.bbcgoodfood.com/recipes/you-it-katsu-curry')

,Title,Rating,Difficulty,Serving,Preptime,Cook_time,Ingredients,Link
0,Katsu curry,4,Easy,4,Prep: 25 mins,Cook: 40 mins,4 tbsp rapeseed oil or vegetable oil Rapeseed ...,https://www.bbcgoodfood.com/recipes/you-it-kat...


In [7]:
dfff = bbc_goodfood_scraper('https://www.bbcgoodfood.com/recipes/you-it-katsu-curry')
dfff.to_csv('DA.csv')

In [8]:
import pandas as pd

# Open the scaped dataframe full of bbcgood food recipies
df = pd.read_csv('C:/Users/robert.lowe/wordlist/food_new.csv')

In [9]:
df = df.join(df['ingredients;https://www.bbcgoodfood.com/recipes;'].str.rsplit("ingredients;", n=1, expand=True))

In [10]:
df.rename(columns={1: 'Website_link'}, inplace=True)

In [11]:
df['Website_link'] = df['Website_link'].astype(str)

In [12]:
def rem(x):
    return x.replace(';', '')

In [13]:
df.Website_link = df.Website_link.apply(rem)

In [14]:
df.head()

,ingredients;https://www.bbcgoodfood.com/recipes;,0,Website_link
0,ingredients;https://www.bbcgoodfood.com/recipes;,,https://www.bbcgoodfood.com/recipes
1,ingredients;https://www.bbcgoodfood.com/recipes;,,https://www.bbcgoodfood.com/recipes
2,ingredients;https://www.bbcgoodfood.com/recipes;,,https://www.bbcgoodfood.com/recipes
3,ingredients;https://www.bbcgoodfood.com/recipes;,,https://www.bbcgoodfood.com/recipes
4,ingredients;https://www.bbcgoodfood.com/recipes;,,https://www.bbcgoodfood.com/recipes


In [15]:
df = df[['Website_link']]
df.head()


,Website_link
0,https://www.bbcgoodfood.com/recipes
1,https://www.bbcgoodfood.com/recipes
2,https://www.bbcgoodfood.com/recipes
3,https://www.bbcgoodfood.com/recipes
4,https://www.bbcgoodfood.com/recipes


In [16]:
df = df.drop_duplicates()

In [17]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 342070 entries, 0 to 2161088
Data columns (total 1 columns):
Website_link    342070 non-null object
dtypes: object(1)
memory usage: 5.2+ MB


In [18]:
def recipe_flag(x):
    if 'https://www.bbcgoodfood.com/recipes' in x:
        return 1
    else:
        return 0

    

In [19]:
df['Website_link_flag'] = df['Website_link'].apply(recipe_flag)

In [20]:
df.head()

,Website_link,Website_link_flag
0,https://www.bbcgoodfood.com/recipes,1
6,https://www.bbcgoodfood.com/recipes/collection...,1
16,https://www.bbcgoodfood.com/recipes/collection...,1
24,https://www.bbcgoodfood.com/recipes/collection...,1
33,https://www.bbcgoodfood.com/recipes/category/s...,1


In [21]:
df = df[df['Website_link_flag'] == 1]

In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
Int64Index: 983 entries, 0 to 52642
Data columns (total 2 columns):
Website_link         983 non-null object
Website_link_flag    983 non-null int64
dtypes: int64(1), object(1)
memory usage: 23.0+ KB


In [23]:
# create a list of recipes for run the bbcgoodfood scraper function on
df1= df.Website_link
df1

0                      https://www.bbcgoodfood.com/recipes
6        https://www.bbcgoodfood.com/recipes/collection...
16       https://www.bbcgoodfood.com/recipes/collection...
24       https://www.bbcgoodfood.com/recipes/collection...
33       https://www.bbcgoodfood.com/recipes/category/s...
41       https://www.bbcgoodfood.com/recipes/category/v...
49       https://www.bbcgoodfood.com/recipes/collection...
56       https://www.bbcgoodfood.com/recipes/collection...
63       https://www.bbcgoodfood.com/recipes/collection...
69       https://www.bbcgoodfood.com/recipes/collection...
75       https://www.bbcgoodfood.com/recipes/collection...
83       https://www.bbcgoodfood.com/recipes/collection...
97       https://www.bbcgoodfood.com/recipes/collection...
103      https://www.bbcgoodfood.com/recipes/category/s...
110      https://www.bbcgoodfood.com/recipes/collection...
116      https://www.bbcgoodfood.com/recipes/collection...
123      https://www.bbcgoodfood.com/recipes/collection.

In [24]:
# loop the bbc goodfood scraper over the list of recipes 
x1 = [bbc_goodfood_scraper(x) for x in df1]

In [25]:
bbc_good_food = pd.concat(x1)

In [26]:
bbc_good_food = bbc_good_food.drop_duplicates()

In [27]:
bbc_good_food.to_csv('bbc_good_food_scrape_2.csv')

In [28]:
bbc_good_food.head()

,Title,Rating,Difficulty,Serving,Preptime,Cook_time,Ingredients,Link
0,Steaks with goulash sauce & sweet potato fries,5,Easy,2,Prep: 10 mins,Cook: 25 mins,"3 tsp rapeseed oil, 250g sweet potatoes, 1 tbs...",https://www.bbcgoodfood.com/recipes/steaks-gou...
0,Lentil soup,5,Easy,4,Prep: 10 mins,Cook: 1 hr,"2l veg or ham stock, 150g red lentils Lentils ...",https://www.bbcgoodfood.com/recipes/1564648/le...
0,"Spaghetti with prawns, chilli & rocket",4,Easy,4,Prep: 5 mins,Cook: 15 mins - 20 mins,"400g spaghetti, 2 long red chillies Chilli chi...",https://www.bbcgoodfood.com/recipes/1125/spagh...
0,Cheat's scampi with chunky chips,4,Easy,4,Prep: 15 mins,Cook: 45 mins,"3 large baking potatoes, 1 tbsp olive oil Oliv...",https://www.bbcgoodfood.com/recipes/1167646/ch...
0,Easy Thai prawn curry,4,Easy,4,Prep: 5 mins,Cook: 15 mins,"1 tbsp vegetable oil, 1 onion, 1 tsp fresh roo...",https://www.bbcgoodfood.com/recipes/7418/easy-...
